In [54]:
import os
from dotenv import load_dotenv
import requests

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
base_url = "http://43.200.7.56:8008/v1"
chat_model = "glm-4-flash"
emb_model = "embedding-3"

model_name = "Qwen2.5-32B-Instruct-AWQ"
openai_api_base = "http://192.168.12.10:8000/v1"  # 本地服务地址


# chat_model = model_name
# base_url=openai_api_base

In [55]:
from openai import OpenAI
from pydantic import Field  # 导入Field，用于Pydantic模型中定义字段的元数据
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    LLMMetadata,
)
from llama_index.core.embeddings import BaseEmbedding


from llama_index.core.llms.callbacks import llm_completion_callback
from typing import List, Any, Generator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings  

from llama_index.embeddings.openai import OpenAIEmbedding

# 定义OurLLM类，继承自CustomLLM基类
class OurLLM(CustomLLM):
    api_key: str = Field(default=api_key)
    base_url: str = Field(default=base_url)
    model_name: str = Field(default=chat_model)
    client: OpenAI = Field(default=None, exclude=True)  # 显式声明 client 字段

    def __init__(self, api_key: str, base_url: str, model_name: str = chat_model, **data: Any):
        super().__init__(**data)
        self.api_key = api_key
        self.base_url = base_url
        self.model_name = model_name
        self.client = OpenAI(api_key=self.api_key, base_url=self.base_url)  # 使用传入的api_key和base_url初始化 client 实例

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            model_name=self.model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        response = self.client.chat.completions.create(model=self.model_name, messages=[{"role": "user", "content": prompt}])
        if hasattr(response, 'choices') and len(response.choices) > 0:
            response_text = response.choices[0].message.content
            return CompletionResponse(text=response_text)
        else:
            raise Exception(f"Unexpected response format: {response}")

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> Generator[CompletionResponse, None, None]:
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            stream=True
        )

        try:
            for chunk in response:
                chunk_message = chunk.choices[0].delta
                if not chunk_message.content:
                    continue
                content = chunk_message.content
                yield CompletionResponse(text=content, delta=content)

        except Exception as e:
            raise Exception(f"Unexpected response format: {e}")

llm = OurLLM(api_key=api_key, base_url=base_url, model_name=chat_model)
llmlocal = OurLLM(api_key=api_key, base_url=openai_api_base, model_name=model_name)



In [ ]:
# 测试对话模型
response = llm.complete("你是谁？")
print(response)

In [ ]:
# 测试对话模型
response = llmlocal.complete("你是谁？")
print(response)

In [ ]:

from llama_index.embeddings.openai import OpenAIEmbedding

emb_model ="bge-m3"
api_key ="23231"
base_url="http://192.168.12.10:9997/v1"


embedding = OpenAIEmbedding(
    api_key = api_key,
    model = emb_model,
    api_base = base_url # 注意这里单词不一样
)

emb = embedding.get_text_embedding("你好呀呀")
Settings.embed_model = embedding
len(emb), type(emb)

In [ ]:
# 测试嵌入模型
# 测试嵌入模型
try:
    emb = Settings.embed_model.get_text_embedding("你好呀呀")
    print(f"Embedding length: {len(emb)}, Type: {type(emb)}")
    print(f"Embedding vector: {emb}")
except Exception as e:
    print(f"Error: {e}")

In [73]:
# 从指定文件读取，输入为List
from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['./docs/问答手册.txt']).load_data()

# 构建节点
from llama_index.core.node_parser import SentenceSplitter
transformations = [SentenceSplitter(chunk_size = 512)]

from llama_index.core.ingestion.pipeline import run_transformations
nodes = run_transformations(documents, transformations=transformations)

# 构建索引
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
from llama_index.core import StorageContext, VectorStoreIndex

emb = embedding.get_text_embedding("你好呀呀")
vector_store = FaissVectorStore(faiss_index=faiss.IndexFlatL2(len(emb)))
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes = nodes,
    storage_context=storage_context,
    embed_model = embedding,
)

In [74]:
# 构建检索器
from llama_index.core.retrievers import VectorIndexRetriever
# 想要自定义参数，可以构造参数字典
kwargs = {'similarity_top_k': 5, 'index': index, 'dimensions': len(emb)} # 必要参数
retriever = VectorIndexRetriever(**kwargs)

# 构建合成器
from llama_index.core.response_synthesizers  import get_response_synthesizer
response_synthesizer = get_response_synthesizer(llm=llmlocal, streaming=True)

# 构建问答引擎
from llama_index.core.query_engine import RetrieverQueryEngine
engine = RetrieverQueryEngine(
      retriever=retriever,
      response_synthesizer=response_synthesizer,
        )

In [ ]:
# 提问
question = "审计法最早是什么时候由谁颁发的？最新的一次修改是什么时候"
response = engine.query(question)
for text in response.response_gen:
    print(text, end="")

In [76]:
# 配置查询工具
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="RAG工具",
            description=(
                "用于在原文中检索相关信息"
            ),
        ),
    ),
]

In [81]:
# 创建ReAct Agent
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(query_engine_tools, llm=llmlocal, verbose=True)

In [84]:
# 让Agent完成任务
# response = agent.chat("请问商标注册需要提供哪些文件？")
response = agent.chat("审计法中对审计数据是怎么描述和授予审计机关权力的？")
print(response)

> Running step 8ecc4722-3312-4ee2-9a65-a9a5e70bd102. Step input: 审计法中对审计数据是怎么描述和授予审计机关权力的？
Thought: 我需要使用RAG工具来检索审计法中对审计数据的描述以及对审计机关权力的授予内容。
Action: RAG工具
Action Input: {'input': '审计法中对审计数据是怎么描述和授予审计机关权力的？'}
Observation: 根据《中华人民共和国审计法》，审计机关在获取和使用审计数据方面拥有以下权力和相关规定：

1. **数据提供权限**：审计机关有权要求被审计单位按其规定提供财务、会计资料以及与财政收支、财务收支有关的业务、管理等资料，包括电子数据和有关文档。被审计单位不得拒绝、拖延、谎报。同时，被审计单位负责人应对所提供资料的及时性、真实性和完整性负责（见第三十四条）。

2. **数据获取方式**：国家政务信息系统和数据共享平台应当按照规定向审计机关开放。如果审计机关通过这些系统和平台取得的电子数据等资料能够满足其需要，那么审计机关不得要求被审计单位重复提供这些资料（见第三十五条）。

3. **数据分析与核实**：审计机关对取得的电子数据等资料进行综合分析，如果需要向被审计单位核实有关情况，被审计单位应当予以配合（见第三十四条）。

这些规定确保了审计机关能够有效获取和使用必要的数据，以履行审计职责。
> Running step 3d3718f3-5811-4183-8907-bef5a948ef3a. Step input: None
Thought: 我可以回答用户的问题，不需要使用更多工具。
Answer: 根据《中华人民共和国审计法》，审计机关在获取和使用审计数据方面有以下权力和相关规定：

1. **数据提供权限**：审计机关有权要求被审计单位提供财务、会计资料以及与财政收支、财务收支有关的业务、管理等资料，包括电子数据和有关文档。被审计单位不得拒绝、拖延或谎报，其负责人应对所提供资料的及时性、真实性和完整性负责。

2. **数据获取方式**：国家政务信息系统和数据共享平台应当按规定向审计机关开放。如果审计机关通过这些系统和平台取得的电子数据等资料能够满足其需要，那么审计机关不得要求被审计单位重复提供这些资料。

3. **数据分